In [9]:
import simpy
import random

# Define constants based on the provided parameters
NUM_CLIENTS = random.randint(100, 200)
NUM_DATACENTERS = 1
NUM_HOSTS = 2

# Cloud node parameters
CLOUD_NODES = 10
CLOUD_CPU_POWER = (3000, 5000)  # MIPS
CLOUD_RAM = 8  # GB
CLOUD_STORAGE = 1  # TB
CLOUD_BANDWIDTH = 0.5  # Gb/s

# Fog node parameters
FOG_NODES = 10
FOG_CPU_POWER = (1000, 2800)  # MIPS
FOG_RAM = 2  # GB
FOG_STORAGE = 0.5  # TB
FOG_BANDWIDTH = 1  # Gb/s

# Task parameters
TASK_LENGTH = (2000, 56000)  # MI
DATA_SIZE = (400, 600)  # MB
TASK_SETS = [300, 600, 900, 1200, 1500]  # Different sets of task numbers

class Node:
    def __init__(self, env, cpu_power, ram, storage, bandwidth):
        self.env = env
        self.cpu_power = cpu_power
        self.ram = ram
        self.storage = storage
        self.bandwidth = bandwidth
        self.queue = simpy.Resource(env, capacity=1)
        self.total_execution_time = 0

    def process_task(self, task_length, data_size):
        # Processing time is based on task length and CPU power
        processing_time = task_length / self.cpu_power
        self.total_execution_time += processing_time
        yield self.env.timeout(processing_time)

    def calculate_energy_consumption(self, makespan):
        beta_j = 10**-8 * self.cpu_power**2
        alpha_j = 0.6 * beta_j
        active_energy = self.total_execution_time * beta_j
        idle_energy = (makespan - self.total_execution_time) * alpha_j
        total_energy = active_energy + idle_energy
        return total_energy

def generate_tasks(env, nodes, task_length_range, data_size_range, num_tasks):
    task_start_times = []
    task_end_times = []
    
    for _ in range(num_tasks):
        task_length = random.randint(*task_length_range)
        data_size = random.randint(*data_size_range)
        node = random.choice(nodes)
        task_start_times.append(env.now)
        env.process(handle_task(env, node, task_length, data_size, task_end_times))
        yield env.timeout(1)  # Small delay to simulate task arrival interval
    
    yield env.timeout(0)  # Yield control to allow all tasks to be processed
    yield env.all_of([env.timeout(1) for _ in range(num_tasks)])  # Wait for all tasks to be completed
    makespan = max(task_end_times) - min(task_start_times)
    return makespan

def handle_task(env, node, task_length, data_size, task_end_times):
    with node.queue.request() as request:
        yield request
        print(f"Task assigned to node with CPU power {node.cpu_power} at time {env.now}")
        yield env.process(node.process_task(task_length, data_size))
        task_end_times.append(env.now)

def main():
    results = {}

    for num_tasks in TASK_SETS:
        env = simpy.Environment()
        cloud_nodes = [
            Node(env, random.randint(*CLOUD_CPU_POWER), CLOUD_RAM, CLOUD_STORAGE, CLOUD_BANDWIDTH)
            for _ in range(CLOUD_NODES)
        ]
        fog_nodes = [
            Node(env, random.randint(*FOG_CPU_POWER), FOG_RAM, FOG_STORAGE, FOG_BANDWIDTH)
            for _ in range(FOG_NODES)
        ]
        all_nodes = cloud_nodes + fog_nodes
        
        makespan = env.process(generate_tasks(env, all_nodes, TASK_LENGTH, DATA_SIZE, num_tasks))
        env.run()
        makespan = makespan.value
        total_energy_consumption = sum(node.calculate_energy_consumption(makespan) for node in all_nodes)
        
        results[num_tasks] = {
            "makespan": makespan,
            "energy_consumption": total_energy_consumption
        }

    # Output the makespan and energy consumption for each set of tasks
    for num_tasks, result in results.items():
        print(f"Makespan for {num_tasks} tasks: {result['makespan']}")
        print(f"Total energy consumption for {num_tasks} tasks: {result['energy_consumption']}")

if __name__ == "__main__":
    main()


Task assigned to node with CPU power 1486 at time 0
Task assigned to node with CPU power 3933 at time 2
Task assigned to node with CPU power 2610 at time 3
Task assigned to node with CPU power 3516 at time 4
Task assigned to node with CPU power 1891 at time 5
Task assigned to node with CPU power 4386 at time 6
Task assigned to node with CPU power 3259 at time 7
Task assigned to node with CPU power 4426 at time 8
Task assigned to node with CPU power 2373 at time 9
Task assigned to node with CPU power 4839 at time 11
Task assigned to node with CPU power 4426 at time 11.880704925440579
Task assigned to node with CPU power 4277 at time 12
Task assigned to node with CPU power 3728 at time 13
Task assigned to node with CPU power 3908 at time 14
Task assigned to node with CPU power 2612 at time 16
Task assigned to node with CPU power 2610 at time 18
Task assigned to node with CPU power 4386 at time 19
Task assigned to node with CPU power 4277 at time 19.4980126256722
Task assigned to node wit